#### **Setup - Install Libraries and Import Dependencies**

In [10]:
!pip install -qqq transformers sentence-transformers faiss-cpu langchain ipywidgets langchain-community PyPDF2

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss # FAISS is used via LangChain's FAISS class
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import warnings
from google.colab import files # For file upload
import PyPDF2

# Suppress specific warnings that might clutter output
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')
warnings.filterwarnings("ignore", category=FutureWarning)

print("Libraries installed and dependencies imported.")

# --- Configuration Parameters ---
# Define chunking parameters
CHUNK_SIZE = 400  # Number of tokens/characters per chunk.
CHUNK_OVERLAP = 40 # Overlap between chunks to maintain context.

# Model names
EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'
LLM_MODEL_NAME = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

# Placeholder for the uploaded document file name
UPLOADED_FILE_NAME = None

Libraries installed and dependencies imported.


#### **Upload Your Assignment Document**

In [11]:
uploaded = files.upload()

# Check if a file was uploaded
if uploaded:
    # Get the name of the first uploaded file
    global UPLOADED_FILE_NAME
    UPLOADED_FILE_NAME = list(uploaded.keys())[0]
    print(f"File '{UPLOADED_FILE_NAME}' uploaded successfully.")
else:
    print("No file was uploaded. Please upload your document.")

Saving 9241544228_eng.pdf to 9241544228_eng.pdf
File '9241544228_eng.pdf' uploaded successfully.


#### **Document Ingestion and Chunking**

In [12]:
if UPLOADED_FILE_NAME is None:
    print("Error: No document file name found. Please run the upload cell first.")
else:
    try:
        if UPLOADED_FILE_NAME.lower().endswith('.pdf'):
            text_content = ""
            with open(UPLOADED_FILE_NAME, "rb") as f:
                reader = PyPDF2.PdfReader(f)
                for page_num in range(len(reader.pages)):
                    text_content += reader.pages[page_num].extract_text()
            print(f"Content from '{UPLOADED_FILE_NAME}' loaded as PDF.")
        else:
            with open(UPLOADED_FILE_NAME, "r", encoding="utf-8") as f:
                text_content = f.read()
            print(f"Content from '{UPLOADED_FILE_NAME}' loaded as text (UTF-8).")


        # Initialize the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
            length_function=len,
            add_start_index=True
        )

        # Create document chunks
        chunks = text_splitter.create_documents([text_content])

        print(f"Original document length: {len(text_content)} characters")
        print(f"Number of chunks created: {len(chunks)}")
        print(f"Example of first chunk content (first 150 chars):\n'{chunks[0].page_content[:150]}...'")
        print(f"Example of first chunk metadata: {chunks[0].metadata}")

    except FileNotFoundError:
        print(f"Error: File '{UPLOADED_FILE_NAME}' not found. Please ensure it was uploaded correctly.")
    except Exception as e:
        print(f"An error occurred during document ingestion: {e}")

Content from '9241544228_eng.pdf' loaded as PDF.
Original document length: 658968 characters
Number of chunks created: 1854
Example of first chunk content (first 150 chars):
'ThelCD-10
Classification
of Mental and
Behavioural
Disorders
Clinical
descriptions
and diagnosticguidelines
| World Health Organization
I Geneva
I 199...'
Example of first chunk metadata: {'start_index': 0}


#### **Embedding and Vector Store**


In [13]:
if 'chunks' not in locals() or not chunks:
    print("Error: No chunks found. Please ensure document ingestion was successful.")
else:
    # Initialize the embedding model
    print(f"Loading embedding model: {EMBEDDING_MODEL_NAME}...")
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
    print("Embedding model loaded.")

    # Create a FAISS vector store from the document chunks and embeddings
    print("Creating FAISS vector store...")
    vector_store = FAISS.from_documents(chunks, embeddings)
    print("FAISS vector store created successfully.")

Loading embedding model: all-MiniLM-L6-v2...
Embedding model loaded.
Creating FAISS vector store...
FAISS vector store created successfully.


#### **LLM Integration - Load and Configure the Language Model**

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the tokenizer for the chosen LLM
print(f"Loading tokenizer for LLM: {LLM_MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME)
print("Tokenizer loaded.")

# Load the LLM itself
print(f"Loading LLM: {LLM_MODEL_NAME}...")
try:
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    ).to(device)
    print("LLM loaded successfully.")

    # Create a Hugging Face pipeline for text generation
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        return_full_text=False,
        pad_token_id=tokenizer.eos_token_id
    )

    # Wrap the Hugging Face pipeline with LangChain's HuggingFacePipeline LLM
    llm = HuggingFacePipeline(pipeline=pipe)
    print("LLM configured via HuggingFacePipeline for LangChain.")

except Exception as e:
    print(f"ERROR: Could not load LLM '{LLM_MODEL_NAME}'. This might be due to insufficient VRAM or model compatibility issues.")
    print(f"Please try a smaller model or ensure you have a GPU runtime. Error details: {e}")
    class DummyLLM:
        def __call__(self, prompt, stop=None):
            print(f"Dummy LLM received prompt:\n{prompt}\n")
            return "Dummy answer: LLM failed to load. Please check your LLM configuration."
    llm = DummyLLM()
    print("Using a Dummy LLM as a fallback. The QA system will not provide real answers.")

Using device: cuda
Loading tokenizer for LLM: TinyLlama/TinyLlama-1.1B-Chat-v1.0...
Tokenizer loaded.
Loading LLM: TinyLlama/TinyLlama-1.1B-Chat-v1.0...


Device set to use cuda:0


LLM loaded successfully.
LLM configured via HuggingFacePipeline for LangChain.


#### **Query Interface and Retrieval-Augmented Generation (RAG) Chain**

In [15]:
if 'vector_store' not in locals():
    print("Error: Vector store not found. Please ensure embedding and vector store creation was successful.")
elif 'llm' not in locals():
    print("Error: LLM not loaded. Please ensure LLM integration was successful.")
else:
    # Configure the retriever
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    print(f"Retriever configured to fetch top {retriever.search_kwargs['k']} chunks.")

    # Set up the RAG chain using LangChain's RetrievalQA
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    print("RAG chain (RetrievalQA) configured successfully.")

Retriever configured to fetch top 3 chunks.
RAG chain (RetrievalQA) configured successfully.


#### **Output - Run Sample Query and Display Results**

In [16]:
if 'qa_chain' not in locals():
    print("Error: QA chain not found. Please ensure the RAG chain was set up successfully.")
else:
    # The specific test question provided in the assignment
    test_question = "Give me the correct coded classification for the following diagnosis: 'Recurrent depressive disorder, currently in remission'"

    print(f"\n--- Test Question ---\n{test_question}")

    # Execute the RAG chain with the test question
    response = qa_chain({"query": test_question})

    # Display the generated answer
    print("\n--- Generated Answer ---")
    print(response["result"])

    # Display the source documents that were used by the LLM to generate the answer
    print("\n--- Source Documents (Context used by LLM) ---")
    if response["source_documents"]:
        for i, doc in enumerate(response["source_documents"]):
            print(f"\nChunk {i+1} (Source Index: {doc.metadata.get('start_index', 'N/A')}):")
            print(doc.page_content)
    else:
        print("No source documents were retrieved for this query.")

    print("\n--- End of Sample Query ---")


--- Test Question ---
Give me the correct coded classification for the following diagnosis: 'Recurrent depressive disorder, currently in remission'

--- Generated Answer ---
 The code for this diagnosis would be F33.1

--- Source Documents (Context used by LLM) ---

Chunk 1 (Source Index: 361532):
currently depressed (F31.3, F31.4 or F31.5), a recurrent depressive
disorder  (F33.  -) or a depressive episode  (F32.  -). At  times,  however,
the criteria for the diagnosis of another mental disorder cannot be
met, although there is often some evidence of a psychopathologicalbasis for the complaint.
Some patients will themselves make the connection between their

Chunk 2 (Source Index: 128960):
The following five-character codes might be used to specify the clinical
disorder:
F06.30 Organic manic disorder
F06.31 Organic bipolar affective disorder
63MENTAL AND BEHAVIOURAL DISORDERS
F06.32 Organic depressive disorder
F06.33 Organic mixed affective disorder
F06.4 Organic anxiety disorder
A d

###Interactive Query Mode

In [17]:
while True:
    user_query = input("\nEnter your question (type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break
    print(f"\n--- Your Question ---\n{user_query}")
    user_response = qa_chain({"query": user_query})
    print("\n--- Generated Answer ---")
    print(user_response["result"])
    print("\n--- Source Documents (Context used by LLM) ---")
    if user_response["source_documents"]:
        for i, doc in enumerate(user_response["source_documents"]):
            print(f"\nChunk {i+1}:")
            print(doc.page_content)
    else:
        print("No source documents were retrieved for this query.")
print("\nInteractive query mode exited.")



--- Interactive Query Mode ---

Enter your question (type 'exit' to quit): What is the purpose of the International Classification of Diseases?

--- Your Question ---
What is the purpose of the International Classification of Diseases?

--- Generated Answer ---
 The purpose of the International Classification of Diseases (ICD) is to provide a standardized set of codes for medical diagnoses and billing procedures. It helps healthcare providers to communicate effectively with each other around the world and improves the efficiency of healthcare systems.

--- Source Documents (Context used by LLM) ---

Chunk 1:
clearly stated ways, and thus to maximize the homogeneity ofstudy groups and the comparability of findings in multicentre and
international studies.
The book, which covers over 300 disorders, is derived from
chapter V(F) of the Tenth Revision of the International Statistical
Classification of Diseases and Related Health Problems (ICD-10).

Chunk 2:
in the Tenth Revision of
 the In